In [2]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import seaborn as sns

In [3]:
TRAIN_DATA_PATH = "data/yellow_tripdata_2022-01.parquet"
VALIDATION_DATA_PATH = "data/yellow_tripdata_2022-02.parquet"

In [18]:
test = pd.read_parquet(TRAIN_DATA_PATH)

In [19]:
test.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
dtype: object

## Data Preparation

In [20]:
def read_data(filepath: str="data/", categorical_fields: [] = ["dropoff_location_id", "pickup_location_id"]) -> pd.DataFrame:
    """
    - Read data by the path
    - Clean data: rename fields, remove outliers, etc
    - Print some information about the figures

    :param filepath: path to parquet files
    :return: DataFrame with cleaned data
    """
    fields = ["tpep_pickup_datetime", "tpep_dropoff_datetime", "PULocationID", "DOLocationID", "trip_distance"]

    yellow_ride_df = pd.read_parquet(filepath)
    yellow_ride_df = yellow_ride_df[fields].copy()
    yellow_ride_df.rename(columns={
        "tpep_pickup_datetime": "pickup_datetime",
        "tpep_dropoff_datetime": "dropoff_datetime",
        "PULocationID": "pickup_location_id",
        "DOLocationID": "dropoff_location_id",
    }, inplace=True)

    yellow_ride_df["pickup_datetime"] = pd.to_datetime(yellow_ride_df.pickup_datetime)
    yellow_ride_df["dropoff_datetime"] = pd.to_datetime(yellow_ride_df.dropoff_datetime)

    yellow_ride_df["duration"] = yellow_ride_df.dropoff_datetime - yellow_ride_df.pickup_datetime
    yellow_ride_df["duration_minutes"] = yellow_ride_df.duration.dt.total_seconds() / 60
    yellow_ride_df.drop(columns=["duration"], inplace=True)


    # Remove trip duration outliers
    lower_duration_bound = yellow_ride_df.duration_minutes.quantile(0.05)
    upper_duration_bound = yellow_ride_df.duration_minutes.quantile(0.95)
    duration_outliers_filter = (yellow_ride_df.duration_minutes >= lower_duration_bound)\
                               & (yellow_ride_df.duration_minutes <= upper_duration_bound)

    records_left_count = len(yellow_ride_df.loc[duration_outliers_filter]) / len(yellow_ride_df) * 100
    duration_std_value_before = yellow_ride_df.duration_minutes.std()

    yellow_ride_df = yellow_ride_df.loc[duration_outliers_filter].copy()
    duration_std_value_after = yellow_ride_df.duration_minutes.std()


    yellow_ride_df[categorical_fields] = yellow_ride_df[categorical_fields].astype(str)
    yellow_ride_df["pickup_dropoff_combination"] = yellow_ride_df.pickup_location_id + yellow_ride_df.dropoff_location_id

    print(f"Standard deviation of duration (minutes): {duration_std_value_before:.2f}")
    print(f"Standard deviation of duration (minutes) without outliers: {duration_std_value_after:.2f}")
    print(f"Records left after removing the outliers: {records_left_count:.2f}")

    return yellow_ride_df

## One-hot encoding

In [28]:
def training_dataset_encoding(train_df: pd.DataFrame, feature_fields: [] = ["pickup_dropoff_combination", "trip_distance"]) -> DictVectorizer:
    """

    :return:
    """
    train_dict = train_df[feature_fields].to_dict(orient="records")

    dv = DictVectorizer()
    x_vector = dv.fit_transform(train_dict)

    return (dv, x_vector)


## Training Model

In [29]:
train_rides_df = read_data(TRAIN_DATA_PATH)
dv, x_train_features = training_dataset_encoding(train_rides_df)

target = "duration_minutes"
y_train = train_rides_df[target].values

lr = LinearRegression()
lr.fit(x_train_features, y_train)

Standard deviation of duration (minutes): 46.45
Standard deviation of duration (minutes) without outliers: 6.48
Records left after removing the outliers: 90.06


LinearRegression()

In [25]:
feature_fields = ["pickup_dropoff_combination", "trip_distance"]
validation_rides_df = read_data(VALIDATION_DATA_PATH)
validation_rides_dict = validation_rides_df[feature_fields].to_dict(orient="records")
x_validation_features = dv.transform(validation_rides_dict)

Standard deviation of duration (minutes): 47.26
Standard deviation of duration (minutes) without outliers: 7.12
Records left after removing the outliers: 90.05


In [26]:
target = "duration_minutes"
y_validation_values = validation_rides_df[target].values
y_prediction_values = lr.predict(x_validation_features)

In [27]:
mean_squared_error(y_validation_values, y_prediction_values, squared=False)

4.485216369240176

In [80]:
sns.histplot(y_train, label="train")
sns.histplot(y_prediction, label="prediction")

Validation RMSE: 6.3868576294229085
